# Understanding Subscription Behaviors

Source: Datacamp

<div style="text-align: center; padding: 20px;">
    <img src="austin-distel-744oGeqpxPQ-unsplash.jpeg" alt="Subscription Renewal Insights for a SaaS Company" width="450"/>
</div>


<!-- Image source - https://unsplash.com/photos/person-using-macbook-pro-744oGeqpxPQ
-->

    
A SaaS company seeks to uncover what drives its clients to renew subscriptions. They’ve collected data on client details, subscription records, and economic indicators and would like to connect them to better understand its clients’ behavior. 

They’ve tasked you with analyzing these datasets to identify the key factors influencing clients’ decisions to renew their subscriptions. 

Your analysis will provide them with insights into which customers are renewing their products and the reasons behind their renewals. The company can leverage these insights to make informed decisions to increase renewal rates and improve customer loyalty, helping them stay competitive and ensure long-term growth.


## The Data

The company have provided you with three datasets for your analysis. A summary of each data is provided below.

## `client_details.csv`

| Column         | Description|
|----------------|---------------------------------------------------------------|
| `client_id`    | Unique identifier for each client. |
| `company_size` | Size of the company (Small, Medium, Large).|
| `industry`     | Industry to which the client belongs (Fintech, Gaming, Crypto, AI, E-commerce).|
| `location`     | Location of the client (New York, New Jersey, Pennsylvania, Massachusetts, Connecticut).|

## `subscription_records.csv`

| Column             | Description   |
|--------------------|---------------|
| `client_id`        | Unique identifier for each client.|
| `subscription_type`| Type of subscription (Yearly, Monthly).|
| `start_date`       | Start date of the subscription - YYYY-MM-DD.|
| `end_date`         | End date of the subscription - YYYY-MM-DD.|
| `renewed`          | Indicates whether the subscription was renewed (True, False).|

## `economic_indicators.csv`

| Column           | Description                                       |
|------------------|---------------------------------------------------|
| `start_date`     | Start date of the economic indicator (Quarterly) - YYYY-MM-DD.|
| `end_date`       | End date of the economic indicator (Quarterly) - YYYY-MM-DD.|
| `inflation_rate` | Inflation rate in the period.|
| `gdp_growth_rate`| Gross Domestic Product (GDP) growth rate in the period.|


In [1]:
# Re-run this cell
# Import required libraries
import pandas as pd

# Import data
client_details = pd.read_csv('data/client_details.csv')
subscription_records = pd.read_csv('data/subscription_records.csv', parse_dates = ['start_date','end_date'])
economic_indicators = pd.read_csv('data/economic_indicators.csv', parse_dates = ['start_date','end_date'])

In [2]:
display(client_details.shape)
display(subscription_records.shape)
display(economic_indicators.shape)

(100, 4)

(100, 5)

(21, 5)

### Client Details Table

In [3]:
# Start coding here, good luck!
client_details.head(3)

,client_id,company_size,industry,location
0,4280387012,Large,Fintech,New York
1,2095513148,Small,Fintech,New Jersey
2,7225516707,Medium,Fintech,Pennsylvania


### Subscription Records Table

In [4]:
subscription_records.head(3)

,client_id,subscription_type,start_date,end_date,renewed
0,1131383004,Yearly,2020-11-11,2021-11-11,False
1,4309371709,Monthly,2021-05-24,2021-06-23,True
2,3183675157,Yearly,2021-12-25,2022-12-25,True


### Economic Indicators Table

In [5]:
if 'Unnamed: 0' in economic_indicators.columns:
    economic_indicators.drop(columns='Unnamed: 0', inplace=True)
    
economic_indicators.head(3)

,start_date,end_date,inflation_rate,gdp_growth_rate
0,2018-01-01,2018-03-31,5.77,3.51
1,2018-04-01,2018-06-30,1.17,2.15
2,2018-07-01,2018-09-30,1.56,1.82


## Question 1
How many total Fintech and Crypto clients does the company have? Store as an integer variable called `total_fintech_crypto_clients`.

In [6]:
total_fintech_crypto_clients = client_details['industry'].isin(['Fintech', 'Crypto']).sum()
print(f'total_fintech_crypto_clients={total_fintech_crypto_clients}')

total_fintech_crypto_clients=47


## Question 2

Which industry has the highest renewal rate? Store as a string variable called `top_industry`.

In [7]:
df_cs = pd.merge(
    left=client_details,
    right=subscription_records,
    how='left',
    on='client_id'
)

df_cs.sample(5)

,client_id,company_size,industry,location,subscription_type,start_date,end_date,renewed
82,4194777577,Large,Fintech,New Jersey,Monthly,2021-12-18,2022-01-17,True
30,5421990790,Large,Fintech,Pennsylvania,Yearly,2018-07-10,2019-07-10,False
78,4760390973,Small,Gaming,New Jersey,Yearly,2022-01-06,2023-01-06,True
4,4387541014,Medium,E-commerce,Massachusetts,Monthly,2018-11-08,2018-12-08,False
34,5217987067,Large,Fintech,New Jersey,Monthly,2019-07-19,2019-08-18,True


In [8]:
df_renewal_stats = df_cs.groupby('industry', as_index=False) \
    .agg(
        num_renewed=('renewed', 'sum'), 
        num_clients=('client_id', 'count')
)

df_renewal_stats['renewal_rate'] = df_renewal_stats['num_renewed'] / df_renewal_stats['num_clients']
df_renewal_stats.sort_values('renewal_rate', ascending=False, inplace=True)

df_renewal_stats

,industry,num_renewed,num_clients,renewal_rate
4,Gaming,16,22,0.727273
0,AI,7,11,0.636364
3,Fintech,12,22,0.545455
2,E-commerce,9,20,0.450000
1,Crypto,11,25,0.440000


In [9]:
top_industry = df_renewal_stats['industry'].iloc[0]
print(f'top_industry={top_industry}')

top_industry=Gaming


## Question 3

For clients that renewed their subscriptions, what was the average inflation rate when their subscriptions were renewed? Store as a float variable called average_inflation_for_renewals.

In [10]:
df_cse = pd.merge_asof(
    left=df_cs.sort_values('end_date'),
    right=economic_indicators,
    left_on='end_date',
    right_on='start_date'
)

df_cse.head()

,client_id,company_size,industry,location,subscription_type,start_date_x,end_date_x,renewed,start_date_y,end_date_y,inflation_rate,gdp_growth_rate
0,4519356806,Small,Gaming,Massachusetts,Monthly,2018-03-04,2018-04-03,True,2018-04-01,2018-06-30,1.17,2.15
1,3683504527,Medium,Gaming,Pennsylvania,Monthly,2018-04-12,2018-05-12,True,2018-04-01,2018-06-30,1.17,2.15
2,7462725203,Medium,E-commerce,Massachusetts,Monthly,2018-05-21,2018-06-20,True,2018-04-01,2018-06-30,1.17,2.15
3,3878940490,Medium,E-commerce,Massachusetts,Monthly,2018-05-23,2018-06-22,False,2018-04-01,2018-06-30,1.17,2.15
4,6751372012,Large,E-commerce,New York,Monthly,2018-05-29,2018-06-28,True,2018-04-01,2018-06-30,1.17,2.15


In [11]:
average_inflation_for_renewals = df_cse[df_cse['renewed']]['inflation_rate'].mean()
print(f'average_inflation_for_renewals={round(average_inflation_for_renewals, 2)}')

average_inflation_for_renewals=4.42
